### Aprior算法实现

算法实现要点：

* 通过pandas的`DataFrame`来实现对`项集（itemset）`的支持度计数
* 项集的集合`C`和`L`都是DataFrame对象

----------
#### Implementation start

In [1]:
import pandas as pd
import numpy as np

def get_C1(data: pd.DataFrame):
    
    C1 = {}
    for col in data:
        for i in data[col]:
            C1.setdefault(i, 0)
            C1[i] = C1[i] + 1

    if np.nan in C1:
        del C1[np.nan]
        
    C1 = pd.DataFrame({'itemset': [set([s]) for s in list(C1.keys())], 'count': list(C1.values())}) #注意这一步将dict的key转化为set的做法
    
    C1 = C1[['itemset','count']]
    
    return C1

In [2]:
def trim_C(C, min_sup):
    
    L =  C[C['count'] >= min_sup]
    L = L[['itemset','count']]
    return L

import itertools

def _findsubsets(s,m):
    return set(itertools.combinations(s, m))


def _connect(L: pd.DataFrame):
    pre_C  = []
    Lkeys = list(L.itemset)
    
    # 需要限制可以进行连接的情况
    lengthOfL = len(Lkeys)
    for i in range(lengthOfL-1):
        listed_i = list(Lkeys[i])
        listed_i.sort()
        for j in range(i+1, lengthOfL):
            listed_j = list(Lkeys[j])
            listed_j.sort()
            if listed_i[:-1] == listed_j[:-1]:
                pre_C.append(Lkeys[i] | Lkeys[j])
    return pre_C


def _remove_candidate_has_infrequent_subset(pre_C: list, k, L_: dict):
    for s in pre_C: 
        for subset in _findsubsets(s, k-1):
            if set(subset) not in list(L_[k-1].itemset):
                if pre_C != []: #防止pre_C已空的情况下,继续remove报错
                    pre_C.remove(s)
    return pre_C


def _count_support(pre_C, D):
    # 创建C的骨架(dataframe)
    C = pd.DataFrame(columns=['itemset', 'count'])
    for i in pre_C:
        C = C.append([{'itemset':i, 'count':0}], ignore_index=True)#非in-place操作,注意赋值回去给C
    
    for index, row in D.iterrows():
        row = list(row)
        for cb in pre_C:
            if cb <= set(row):
                C.loc[C.itemset==cb, 'count'] += 1
    return C


# 由L1生成C2
#apriori_gen
def L2C(L: pd.DataFrame, D: pd.DataFrame, k: int, L_: list):
    
    # 1. 连接
    pre_C = _connect(L)

    #2. 剪枝, 删除非频繁候选
    pre_C = _remove_candidate_has_infrequent_subset(pre_C, k, L_)
     
    #3. 支持度计数
    C= _count_support(pre_C, D)
    
    return C

In [3]:
def confidence(L: pd.DataFrame, L_):
    itemset_ses = L['itemset']
    k = len(itemset_ses[0])
    conf_df = pd.DataFrame()
    for itemset in itemset_ses:        
        for ik in range(1, k):
            subsets = _findsubsets(itemset, ik)
            for subset in subsets:
                subset = set(subset)
                diffset = itemset - subset #求出差集
                c_itemset = L.loc[L['itemset'] == itemset, 'count'].values[0]
                _L = L_[ik]
                c_subset = _L.loc[_L['itemset'] == subset, 'count'].values[0]
                conf = c_itemset / c_subset
                conf_df= conf_df.append(
                    {'itemset': itemset, 'start': subset, 'subset_count': c_subset,  'end': diffset, 'itemset_count': c_itemset, 'conf': "{0:.0f}%".format(conf*100)},
                    ignore_index=True)
                # conf为numpy.float64对象
                # python内置round()函数针对numpy.float64不能正确工作，方法是先将numpy.float64转换为python原生的float
    conf_df = conf_df[['itemset', 'start','end', 'subset_count',  'itemset_count', 'conf']]
    return conf_df

#### Implementation end

--------------

In [4]:
inputfile = '../PPDAM/chapter5/demo/data/DMCT_menu_orders.xls'

data = pd.read_excel(inputfile, header=None)

In [5]:
C1 = get_C1(data)
L1 = trim_C(C1,2)
#L1 = L1[['itemset','count']] #调整L1列的顺序, 顺序调整: count itemset -> itemset count
L_ = {1: L1}
C_ = {1: C1}
k = 1

while not L_[k].empty:
    #print(L_[k])
    C_[k+1] = L2C(L_[k], data, k+1, L_)
    L_[k+1] = trim_C(C_[k+1], 2) #指定最小支持度计数(min_sup)为2
    k += 1

In [6]:
from IPython.display import display
for i in L_:
    display(L_[i])

,itemset,count
0,{I1},6
1,{I2},7
2,{I4},2
3,{I3},6
4,{I5},2


,itemset,count
0,"{I1, I2}",4
2,"{I1, I3}",4
3,"{I1, I5}",2
4,"{I4, I2}",2
5,"{I3, I2}",4
6,"{I5, I2}",2


,itemset,count
0,"{I1, I3, I2}",2
1,"{I1, I5, I2}",2


,itemset,count


In [7]:
L2 = L_[2]

In [8]:
L2 = L_[2]

In [9]:
L2

,itemset,count
0,"{I1, I2}",4
2,"{I1, I3}",4
3,"{I1, I5}",2
4,"{I4, I2}",2
5,"{I3, I2}",4
6,"{I5, I2}",2


In [10]:
confidence(L2, L_)

,itemset,start,end,subset_count,itemset_count,conf
0,"{I1, I2}",{I2},{I1},7.0,4.0,57%
1,"{I1, I2}",{I1},{I2},6.0,4.0,67%
2,"{I1, I3}",{I1},{I3},6.0,4.0,67%
3,"{I1, I3}",{I3},{I1},6.0,4.0,67%
4,"{I1, I5}",{I5},{I1},2.0,2.0,100%
5,"{I1, I5}",{I1},{I5},6.0,2.0,33%
6,"{I4, I2}",{I2},{I4},7.0,2.0,29%
7,"{I4, I2}",{I4},{I2},2.0,2.0,100%
8,"{I3, I2}",{I2},{I3},7.0,4.0,57%
9,"{I3, I2}",{I3},{I2},6.0,4.0,67%


In [11]:
def sort_by_conf(df: pd.DataFrame):
    return df.reindex(index=df.conf.str.rstrip('%').astype(float).sort_values(ascending=False).index)


L3 = L_[3]
df = confidence(L3, L_)
df = sort_by_conf(df)

In [12]:
df

,itemset,start,end,subset_count,itemset_count,conf
11,"{I1, I5, I2}","{I5, I2}",{I1},2.0,2.0,100%
9,"{I1, I5, I2}","{I1, I5}",{I2},2.0,2.0,100%
6,"{I1, I5, I2}",{I5},"{I1, I2}",2.0,2.0,100%
10,"{I1, I5, I2}","{I1, I2}",{I5},4.0,2.0,50%
5,"{I1, I3, I2}","{I1, I3}",{I2},4.0,2.0,50%
4,"{I1, I3, I2}","{I1, I2}",{I3},4.0,2.0,50%
3,"{I1, I3, I2}","{I3, I2}",{I1},4.0,2.0,50%
8,"{I1, I5, I2}",{I1},"{I5, I2}",6.0,2.0,33%
2,"{I1, I3, I2}",{I3},"{I1, I2}",6.0,2.0,33%
1,"{I1, I3, I2}",{I1},"{I3, I2}",6.0,2.0,33%


In [13]:
sort_by_conf(df[df['itemset'] == set(['I1','I2','I5'])])

,itemset,start,end,subset_count,itemset_count,conf
6,"{I1, I5, I2}",{I5},"{I1, I2}",2.0,2.0,100%
9,"{I1, I5, I2}","{I1, I5}",{I2},2.0,2.0,100%
11,"{I1, I5, I2}","{I5, I2}",{I1},2.0,2.0,100%
10,"{I1, I5, I2}","{I1, I2}",{I5},4.0,2.0,50%
8,"{I1, I5, I2}",{I1},"{I5, I2}",6.0,2.0,33%
7,"{I1, I5, I2}",{I2},"{I1, I5}",7.0,2.0,29%


### FP-Growth算法实现

步骤1. 第一次扫描事物数据库D，得到频繁1项集L，其中itemset按照计数递减排列

In [14]:
L = C_[1].sort_values(by='count', ascending=False)
L

,itemset,count
1,{I2},7
0,{I1},6
3,{I3},6
2,{I4},2
4,{I5},2


In [15]:
order =  [list(s)[0]  for s in L.itemset]

步骤2. 第二次扫描数据库D，构造`FP tree`和`headTable`

In [16]:
data

,0,1,2,3
0,I1,I2,I5,NaN
1,I2,I4,NaN,NaN
2,I2,I3,NaN,NaN
3,I1,I2,I4,NaN
4,I1,I3,NaN,NaN
5,I2,I3,NaN,NaN
6,I1,I3,NaN,NaN
7,I1,I2,I3,I5
8,I1,I2,I3,NaN


In [17]:

for r in data.iterrows():
    # r是一个tuple，第一项是该行Index，第二行是该行内容的Series
    print(
        sorted([i for i in r[1].dropna()], key=lambda v: order.index(v))
    )

['I2', 'I1', 'I5']
['I2', 'I4']
['I2', 'I3']
['I2', 'I1', 'I4']
['I1', 'I3']
['I2', 'I3']
['I1', 'I3']
['I2', 'I1', 'I3', 'I5']
['I2', 'I1', 'I3']


#### Implementation start

credits: https://blog.csdn.net/gamer_gyt/article/details/51113753

------------

In [18]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        self.tree = ''
 
    def inc(self, numOccur):
        self.count += numOccur
 
    def disp(self, ind=0): #ind: indentation
        
        print('*' * ind, self.name, ':', self.count)
        
        for child in self.children.values():
            child.disp(ind + 1)
            
    
    def stat(self, stat_list: list, ind=0): #ind: indentation
        
        if self.name != '{null}':
            stat_list.append(['*' * ind+self.name, self.count])
        
        for child in self.children.values():
            child.stat(stat_list, ind + 1)

        return stat_list

In [19]:
def createTree(dataSet, minSup=2):
    ''' 创建FP树 '''

    headerTable = {}
    for trans, count in dataSet.items():
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + count

    for k in list(headerTable.keys()):
        if headerTable[k] < minSup:
            del(headerTable[k])
    
    freqItemSet = set(headerTable.keys())
    
    if len(freqItemSet) == 0:
        return None, None
    
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
        
    retTree = treeNode('{null}', 1, None) 
     
    for tranSet, count in dataSet.items(): 
        for c in range(count): #这一步需要多注意
            localD = {}
            for item in tranSet: 
                if item in freqItemSet:
                    localD[item] = headerTable[item][0] 
            if len(localD) > 0:
                orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)] # 排序
                updateTree(orderedItems, retTree, headerTable, 1) # 更新FP树
    return retTree, headerTable

In [20]:
def updateTree(orderedItems, inTree, headerTable, count):
    '''
    做的事情：
    把排序后列表[p|P]的第一项(p)添加到已有的FP tree上面，分两种情况：
        1. 根节点下面已经有p，则根节点下面的p节点计数增加1。
        2. 根节点下面没有p，那么在根节点下面创建新节点p。因为新创建该节点，需要更新headerTable中该项的nodeLink.
    对剩下的P递归调用上面的过程，直到P为空
    '''
    if orderedItems[0] in inTree.children:
        inTree.children[orderedItems[0]].inc(count)
    else:
        inTree.children[orderedItems[0]] = treeNode(orderedItems[0], count, inTree)
        if headerTable[orderedItems[0]][1] == None:
            headerTable[orderedItems[0]][1] = inTree.children[orderedItems[0]]
        else:
            updateHeader(headerTable[orderedItems[0]][1], inTree.children[orderedItems[0]])
            
    if len(orderedItems) > 1:
        updateTree(orderedItems[1:], inTree.children[orderedItems[0]], headerTable, count)

In [21]:
def updateHeader(nodeToUpdate, targetNode):
    while (nodeToUpdate.nodeLink != None):
        nodeToUpdate = nodeToUpdate.nodeLink
    nodeToUpdate.nodeLink = targetNode

In [22]:
def loadSimpDat(data):
    lines = []
    for r in data.iterrows():
        lines.append([i for i in r[1].dropna()])
    input_data = _pre_process(lines)
    return input_data

In [23]:
def _pre_process(lines):
    input_data = {}
    for line in lines:
        s = frozenset(line)
        input_data[s] = input_data.get(s, 0) + 1
    return input_data

In [24]:
simpDat = loadSimpDat(data)
myTree, myTable = createTree(simpDat, 2)

In [25]:
def findPrefixPath(treeNode): #indPrefixPath is also "find conditional base"
    ''' 创建前缀路径 '''
    condPats = {} #条件基是一个字典, key是前缀路基(条件基), value是对应的计数
    while treeNode != None:
        prefixPath = []
        move_to_top_and_record(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        
        treeNode = treeNode.nodeLink

    return condPats

In [26]:
def move_to_top_and_record(leafNode, prefixPath):
    if leafNode.parent != None: #当leftNode是根节点(null)时, 路径记录终止,记录不含根节点
        prefixPath.append(leafNode.name)
        move_to_top_and_record(leafNode.parent, prefixPath)

In [27]:
def parse_tree_to_branches(inTree: list):
    
    tree = [[i[0].count('*') ,i[0].lstrip('*'), i[1]] for i in inTree]
    originTree = [[i[0].lstrip('*'), i[1]] for i in inTree]
    ct = 0
    new_tree = []
    for line  in tree:
        new_tree.append([ct] + line)
        ct += 1
    tree = new_tree
    
    length = len(tree)
    
    level_to_index_list = {}
    
    for line in tree:
        level_to_index_list.setdefault(line[1], [])
        level_to_index_list[line[1]].append(line[0])
    
    levelDescendList = sorted(level_to_index_list.keys(), reverse=True)
    
    lenOflDL = len(levelDescendList)
    
    flat_tree_index = []
    
    for levelNum in levelDescendList:
        for item in level_to_index_list[levelNum]:
            branch = [item]
            for beforeLevelNum in range(1, levelNum):
                localL = []
                for beforeItem in level_to_index_list[beforeLevelNum]:
                    
                    if beforeItem < branch[0]:
                        localL.append(beforeItem)                        
                branch = [max(localL)] + branch
            
            if flat_tree_index != []:
                flag = True
                for testItem in flat_tree_index:
                    if  set(branch) <= set(testItem):
                        flag = False
                if flag:
                    flat_tree_index.append(branch)
            else:
                flat_tree_index.append(branch)
    
    flat_tree = []
    for line in flat_tree_index:
        lineL  = []
        for item in line:
            lineL.append(originTree[item])
        flat_tree.append(lineL)
        
    return flat_tree

In [28]:
import pandas as pd
pd.options.display.max_colwidth = 300

def mineTree(inTree, headerTable, minSup, preFix):

    resultTable = pd.DataFrame(columns=['item', 'CPB', 'Contional FP-tree', 'Frequent Pattern'])

    global_L = [item[0] for item in sorted(list(headerTable.items()), key=lambda p: p[1][0])] #注意headerTable的构造
    # gloabl_L的排序根据支持度从小到大排序

    freqItemDict = {}
    
    for element in global_L: #1 element
        newFreqSet = preFix.copy()
        newFreqSet.add(element)
        
        condPattBases = findPrefixPath(headerTable[element][1]) #2 conditional base

        myCondTree, myHead = createTree(condPattBases, minSup) #创建一个条件FP树 #D的格式问题这里需要解决

        
        if myHead != None: #当element为I2时, myHead为None
            s = []
            myCondTree.stat(stat_list=s)
            flat_tree = parse_tree_to_branches(s)
        
            FP = {}
            for branch in flat_tree: #3. conditional tree
                # [['I1', 4], ['I2', 2]] branch
                localDict = dict(branch)
                baseSet  = set(localDict.keys())
                # baseSet = {'I1','I2'}
                lenOfbaseSet = len(baseSet)
                
                for ik in range(1, lenOfbaseSet+1):
                    subSets = _findsubsets(baseSet, ik)
                    for subSet in subSets:
                        fp = frozenset(set(subSet) | set([element]))
                        FP[fp] = FP.get(fp, 0) + min([localDict[ifk] for ifk in subSet])           
            
            freqItemDict[element]  = FP #4. Frequent Pattern

            #警告: append非原地操作
            resultTable = resultTable.append({'item': element, 'CPB': condPattBases, 'Contional FP-tree':  flat_tree, 'Frequent Pattern': FP}, ignore_index=True)

    return freqItemDict, resultTable
    
                    
def _findsubsets(s,m):
    return set(itertools.combinations(s, m))

In [29]:
def fpGrowth(dataSet, minSup=2):

    myFPtree, myHeaderTab = createTree(dataSet, minSup)
    freqItemDict, resultTable = mineTree(myFPtree, myHeaderTab, minSup, set([]))
    return freqItemDict,resultTable

In [30]:
dataSet = loadSimpDat(data)
freqItems,resultTable = fpGrowth(dataSet)

In [31]:
resultTable

,item,CPB,Contional FP-tree,Frequent Pattern
0,I5,"{('I1', 'I2'): 1, ('I1', 'I3', 'I2'): 1}","[[[I1, 2], [I2, 2]]]","{('I5', 'I2'): 2, ('I1', 'I5'): 2, ('I1', 'I5', 'I2'): 2}"
1,I4,"{('I2'): 1, ('I1', 'I2'): 1}","[[[I2, 2]]]","{('I4', 'I2'): 2}"
2,I1,{('I2'): 4},"[[[I2, 4]]]","{('I1', 'I2'): 4}"
3,I3,"{('I2'): 2, ('I1'): 2, ('I1', 'I2'): 2}","[[[I1, 4], [I2, 2]], [[I2, 2]]]","{('I3', 'I2'): 4, ('I1', 'I3'): 4, ('I1', 'I3', 'I2'): 2}"


In [32]:
from pprint import pformat

In [33]:
print(pformat(freqItems))

{'I1': {frozenset({'I1', 'I2'}): 4},
 'I3': {frozenset({'I3', 'I2'}): 4,
        frozenset({'I1', 'I3'}): 4,
        frozenset({'I1', 'I3', 'I2'}): 2},
 'I4': {frozenset({'I4', 'I2'}): 2},
 'I5': {frozenset({'I5', 'I2'}): 2,
        frozenset({'I1', 'I5'}): 2,
        frozenset({'I1', 'I5', 'I2'}): 2}}


#### Implementation end

------------